In [1]:
import pandas as pd
import geopandas as gpd
import shapely
import os

In [2]:
#CARGO CARTOGRAFIA NACIONAL NO OFICIAL
pais = gpd.GeoDataFrame.from_file('/home/pipe/Downloads/paisNoOficial/rad_paisp.shp')
pais['LINK'] = pais.PROV + pais.DEPTO + pais.FRAC + pais.RADIO
pais.loc[pais.PROV=='06'].head()
pais.drop([u'DEPTO', u'FRAC', u'PERIMETER', u'PROV', u'RADIO', u'RADPAIS_',
       u'RADPAIS_ID'],axis=1,inplace=True)
pais = pais.loc[pais.TIPO!='R',:]
pais.head()


,AREA,LINK,TIPO,geometry
0,207407200.0,380770101,M,"POLYGON ((3470906.5 7569104.5, 3470872.5 75691..."
2,110506900.0,380770202,M,"POLYGON ((3495669.75 7577169.5, 3499717.5 7576..."
3,151297.6,380770103,U,"POLYGON ((3494430.25 7574442.5, 3494745 757444..."
4,462011900.0,380770302,M,"POLYGON ((3470906.5 7569104.5, 3471078.25 7569..."
6,548228900.0,380770303,M,"POLYGON ((3480192.25 7567800.5, 3480257.75 756..."


## load density

In [3]:
#esta data fue descargada desde la pagina oficial de indec 
#http://200.51.91.245/argbin/RpWebEngine.exe/PortalAction?&MODE=MAIN&BASE=CPV2010B&MAIN=WebServerMain.inl
#obteniendo listado por areas a nivel de radio censal
#sexo, con total de filas, para: 

#los 24 partidos del Gran Benos Aires
pbaPob = pd.read_csv('../dt/poblacionPBAcenso.csv')
pbaPob.LINK = pbaPob.LINK.map(lambda x: '0'+str(x))
print pbaPob.shape

#CABA 
cabaPob = pd.read_csv('../dt/poblacionCABAcenso.csv')
cabaPob.LINK = cabaPob.LINK.map(lambda x: '0'+str(x))
print cabaPob.shape

poblacion = cabaPob.append(pbaPob)
print poblacion.shape

(9166, 2)
(3553, 2)
(12719, 2)


In [4]:
agba = pais.merge(poblacion, on='LINK')


In [6]:
agba.loc[agba.LINK.duplicated(),:]

,AREA,LINK,TIPO,geometry,Pob
1824,18459.31,020130104,,"POLYGON ((4192345.75 6152786, 4192385.5 615269...",763
2825,22525.75,020121607,,"POLYGON ((4190128.75 6149061.5, 4190180.75 614...",754


In [7]:
agba['densidad'] = agba.Pob / agba.AREA
agba['densidad'] = pd.qcut(agba.densidad,10,labels=False)
agba = agba.to_crs(epsg=22185)


## Load OSM

In [8]:
#download data
osmFile = os.path.expanduser('~')+'/Downloads/buenos-aires_argentina.osm2pgsql-shapefiles/buenos-aires_argentina_osm_line.shp'
if not os.path.isfile(osmFile):
    os.sys('wget https://s3.amazonaws.com/metro-extracts.mapzen.com/buenos-aires_argentina.osm2pgsql-shapefiles.zip')
    os.sys('unzip https://s3.amazonaws.com/metro-extracts.mapzen.com/buenos-aires_argentina.osm2pgsql-shapefiles.zip')



In [9]:
calles = gpd.read_file('/home/pipe/Downloads/buenos-aires_argentina.osm2pgsql-shapefiles/buenos-aires_argentina_osm_line.shp')
calles = calles.loc[:,['highway','geometry']]
mask =  (calles.highway == 'residential') | (calles.highway == 'unclassified') |  (calles.highway == 'living_street') 
calles = calles.loc[mask,:]
calles = calles.to_crs(epsg=22185)


In [10]:
agba['diss'] = 1

In [11]:
agba.crs

{'init': 'epsg:22185', 'no_defs': True}

In [12]:
agbaLimites = agba.loc[:,['geometry','diss']].dissolve(by='diss')
agbaLimites

,geometry
diss,
1,"POLYGON ((5618811.292373296 6136903.102709887,..."


In [21]:
agbaLim = gpd.GeoDataFrame(agba.groupby("diss").geometry.agg(shapely.ops.unary_union).reset_index(), geometry="geometry", crs=agba.crs)

In [30]:
calles['buffer'] = calles.geometry.buffer(70)
calles.drop(['geometry'],axis=1,inplace=True)
calles.columns = [u'highway', u'geometry']

In [ ]:
res_union = gpd.overlay(calles, agbaLim, how='intersection')

In [ ]:
res_union2 = gpd.overlay(calles, agba, how='intersection')

In [ ]:
res_union.to_file('clip')
res_union2.to_file('clip2')

In [ ]:
#esto es en shapely
objet.intersects(other)

In [14]:
agbaLim.to_file('agbaLim')

In [15]:
calles.to_file('calles')

In [5]:
agba.to_file('caca')

In [16]:
#clip las calles con el limite de agba

In [18]:
#corto los segmentos de calle con cada radio

In [17]:
#spatial join asigno a cada calle el radio que le corresponde

In [19]:
#creo un buffer para cada segmento de calle 

In [20]:
#disuelvo el buffer con LINK como campo 